# Exercise: practice SQLAlchemy 🧪

Let's configure our first RDS instance on AWS and then use SQLAlchemy to write into our remote database and make some SQL queries!

1. Follow the instructions/videos from yesterday (M03-D03/04-Amazon-RDS.ipynb) to create your own RDS instance on AWS 
2. Download <a href="https://www.pgadmin.org/download/" target="_blank">PGAdmin</a> and configure it to access your remote database

If you get stuck at some step, don't hesitate to ask for help to your classmates, your teacher or your TA 🤗.

**Before continuing, please execute the cell below, this will install a package that is required to access your remote database from this notebook:**

In [2]:
from box import Box
import yaml # Only required for different loaders
import pandas as pd

In [2]:
#!pip install psycopg2-binary

## Upload configuration file

In [3]:
conf = Box.from_yaml(filename="./config.yaml")

## 3. Create an sqlalchemy engine that is connected to your AWS RDS instance

In [4]:
from sqlalchemy import create_engine

In [5]:
engine = create_engine(f'postgresql+psycopg2://{conf.userRDS}:{conf.mdp}@{conf.endpoint}')

In [6]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

4. Create a new table named `customers` in your remote database and insert the following data :

| id | name       | country        | job        | age |
|----|------------|----------------|------------|-----|
| 1  | Sauerkraut | Germany        | engineer   | 37  |
| 2  | Jones      | United Kingdom | journalist | 52  |
| 3  | Dupont     | France         | dancer     | 25  |

Optionnal: Use PGAdmin to check that the table has been created without any mistake

In [8]:
# Create table here
from sqlalchemy import Column, Integer, String , Float 

class Customer2(Base):
    __tablename__ = "customers_2"

    id = Column(Integer, primary_key=True)
    name = Column(String)
    country = Column(String)
    job = Column(String)
    age = Column(Integer)

    def __repr__(self):
        return "<Customers(name='{}', country='{}', job='{}, age={} ')>".format(self.name, self.country, self.job, self.age)

In [9]:
Base.metadata.create_all(engine)

In [10]:
# Create instances of Customer
customer_1 = Customer2(id=2, name='Berrettini', country='Italy', job="tenis man", age=25)

# Access Full row 
print(customer_1)

# Access ed_user name 
name = customer_1.name
print(f"name: {name}")

# Access ed_user nickname
country = customer_1.country 
print(f"country: {country}")

<Customers(name='Berrettini', country='Italy', job='tenis man, age=25 ')>
name: Berrettini
country: Italy


In [11]:
# Initialize a sessionmaker 
from sqlalchemy.orm import sessionmaker 
Session = sessionmaker(bind=engine)

# Instanciate Session 
session = Session()

In [12]:
# Add values to db 
session.add(customer_1)

# Commit the results 
session.commit()

5. Execute the cell below to download the famous iris dataset:

In [13]:
species = {0:'setosa', 1: 'versicolor', 2: 'virginica'}

In [16]:
from sklearn import datasets
iris_df, targets = datasets.load_iris(return_X_y=True, as_frame=True)

In [17]:
iris_df["species"] = targets

In [18]:
iris_df.species = iris_df.species.apply(lambda x: species[x])

In [19]:
iris_df.columns = ["sepal_length_cm", "sepal_width_cm", "petal_length_cm", "petal_width_cm", "species"]

In [20]:
iris_df.head(5)

,sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [21]:
class Iris(Base):
    __tablename__ = "iris_2"

    id = Column(Integer, primary_key=True)
    sepal_length_cm = Column(Float)
    sepal_width_cm = Column(Float)
    petal_length_cm = Column(Float)
    petal_width_cm = Column(Float)
    species = Column(String)

    def __repr__(self):
        return "<Iris(sepal_length_cm='{}', sepal_width_cm='{}', petal_length_cm='{}, petal_width_cm={} ')>".format(self.sepal_length_cm, self.sepal_width_cm, self.petal_length_cm, self.petal_width_cm)

In [22]:
Base.metadata.create_all(engine)

6. Create a table in your remote database containing the information of the dataset:

In [23]:
# Add values to db 
iris_df.to_sql(name="iris_2", con=engine, if_exists='append', index=True, index_label='id')


### Now let's make some SQL requests !

To answer the following questions, don't hesitate to refer to <a href="https://www.sqltutorial.org/sql-cheat-sheet/" target="_blank">this cheatsheet</a>. 😉

7. What are the different species present in this dataset?

In [24]:
from sqlalchemy.sql import text
from sqlalchemy import select
# Create a statement 

In [74]:
session2 = Session()

In [75]:
result = session2.query(Iris)
print(result)


SELECT iris_2.id AS iris_2_id, iris_2.sepal_length_cm AS iris_2_sepal_length_cm, iris_2.sepal_width_cm AS iris_2_sepal_width_cm, iris_2.petal_length_cm AS iris_2_petal_length_cm, iris_2.petal_width_cm AS iris_2_petal_width_cm, iris_2.species AS iris_2_species 
FROM iris_2


## Different species present in this dataset

In [73]:
#session2.query(Iris.species).distinct(Iris.species).execute()
session.execute(session.query(Iris.species).distinct(Iris.species)).all()

[('setosa',), ('versicolor',), ('virginica',)]

In [60]:
session.query(Iris.species).distinct(Iris.species)

In [70]:
result = session.execute(select(Iris.species).distinct(Iris.species))

In [72]:
result.all()

[('setosa',), ('versicolor',), ('virginica',)]

8. What is the average sepal length among all species?

In [77]:
from sqlalchemy.sql.expression import func

In [80]:
session.execute(func.avg(Iris.sepal_length_cm)).all()

[(5.843333333333335,)]

9. What is the average sepal length for each species?

In [85]:
session.execute(select(Iris.species, func.avg(Iris.sepal_length_cm)).group_by(Iris.species)).all()

[('virginica', 6.587999999999998),
 ('versicolor', 5.936),
 ('setosa', 5.005999999999999)]

10. How many samples of 'virginica' have sepal length < 6?

In [93]:
session.execute(select(Iris.species, Iris.sepal_length_cm).where(Iris.species == 'virginica', Iris.sepal_length_cm < 6)).all()

[('virginica', 5.8),
 ('virginica', 4.9),
 ('virginica', 5.7),
 ('virginica', 5.8),
 ('virginica', 5.6),
 ('virginica', 5.8),
 ('virginica', 5.9)]

11. For each species, count the number of samples having sepal length < 6:

In [97]:
session.execute(select(Iris.species, func.count(Iris.species)).where(Iris.sepal_length_cm < 6).group_by(Iris.species)).all()

[('virginica', 7), ('versicolor', 26), ('setosa', 50)]